# Stream and Block Ciphers

This notebook implements XXXX

### Stream Ciphers

TODO: What is a stream cipher

#### A simple (unsecure) stream cipher

In [39]:
import binascii

In [40]:
def crand(seed):
    r=[]
    r.append(seed)
    for i in range(30):
        r.append((16807*r[-1]) % 2147483647)
        if r[-1] < 0:
            r[-1] += 2147483647    
    for i in range(31, 34):
        r.append(r[len(r)-31])
    for i in range(34, 344):
        r.append((r[len(r)-31] + r[len(r)-3]) % 2**32)

    while True:
        next = r[len(r)-31]+r[len(r)-3] % 2**32
        r.append(next)
        yield (next >> 1 if next < 2**32 else (next % 2**32) >> 1)

In [127]:
def encrypt(msg, key):
 
    # Convert plain text in (byte) form to hex.
    hexPlain = binascii.hexlify(plainText)
    
    # Convert key to hex. In the code below the key
    # is a list. The map function passes the elements of the list
    # into the lambda function. The join assembles it as one string.
    # Remember the map is an iterable.
    keyHex = "".join(map(lambda x: format(x, 'x')[-6:], key))
    
    # Apply XOR operation (OTP like). To do this need to convert to int.
    cipherInt = int(hexPlain, 16) ^ int(keyHex, 16)
    
    # Return cipher text as hex.
    cipherHex = format(cipherInt, 'x')
    
    return cipherHex

In [128]:
def decrypt(cipherHex, key):

    # Convert the hex back to an int for xor operation.
    cipherInt = int(cipherHex, 16)

    # Convert key to hex. In the code below the key
    # is a list. The map function passes the elements of the list
    # into the lambda function. The join assembles it as one string.
    # Remember the map is an iterable.
    keyHex = "".join(map(lambda x: format(x, 'x')[-6:], key))

    # Decrypt by applying XOR operation again.
    msgHex = format(cipherInt ^ int(keyHex, 16), 'x')

    # Convert back to plain text.
    msgPlain = binascii.unhexlify(msgHex)

    return msgPlain

In [129]:
# Define the message to send.
plainText = b"Hello World Ending"
# Create the key.
seed = 2018
mygen = crand(seed)
key = [next(mygen) for i in range(4)]
print("The key is {}".format(key))

# Encrypt the message.
cipherHex = encrypt(plainText, key)

# Decrypt the message.
msgDecrypt = decrypt(cipherHex, key)
print(msgDecrypt)

The key is [1471611625, 1204518815, 463882823, 963005816]
b'Hello World Ending'


In [125]:
# Create new key.
seed = 54321
mygen = crand(seed)
key = [next(mygen) for i in range(6)] # NB MUST BE SAME LENGTH AS CIPHER HEX FOR OTP!!!!

print("The key is {}".format(key))

# Defined cipher text.
cipherHex = 'e5d8443c6ac32d3ee5c7398ecf7f9e03f619'

# Decrypt the message.
msgDecrypt = decrypt(cipherHex, key)
print(msgDecrypt)

The key is [60209456, 357898661, 257185675, 1235229180, 765860086, 1920452902]
b'satisfying right??'


In [130]:
import os

In [131]:
os.urandom(6)

b',\xc2Pg\xde8'